In [2]:
# Import our dependencies
import pandas as pd
from path import Path
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist

C:\Users\Seva\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Seva\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Seva\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Seva\anaconda3\envs\PythonData\lib\site-packages\ten

In [41]:
# Import our clean dataset
file_path = Path("../clean_data/combined_olympic_data.csv")
df = pd.read_csv(file_path)
df.head(10)

,country_name,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,total
0,Afghanistan,33370794,613.856689,0.465,0.676,12,1
1,Albania,2889104,4578.631994,0.733,0.267,33,0
2,United Arab Emirates,9214175,43751.838890,0.835,0.247,70,0
3,Argentina,42669500,12334.798250,0.836,0.364,34,4
4,Armenia,2912403,3986.231624,0.733,0.299,37,2
5,Australia,23475686,62510.791170,0.935,0.123,80,38
6,Austria,8546356,51717.495940,0.885,0.083,72,17
7,Azerbaijan,9535079,7891.313147,0.751,0.330,29,9
8,Burundi,9844297,274.857948,0.400,0.483,20,0
9,Belgium,11209057,47700.540360,0.890,0.076,76,3


In [42]:
df["total"].value_counts()

0      75
1      18
2       8
3       6
4       5
6       5
8       4
17      3
5       3
9       3
12      3
13      3
7       2
15      2
18      2
38      2
100     1
19      1
16      1
21      1
23      1
31      1
36      1
43      1
44      1
46      1
50      1
63      1
70      1
97      1
132     1
Name: total, dtype: int64

In [43]:
bin_labels = ['0', '1-5', '6-10', '11-15', '16-20', '21-30', '31-40', '41-50', '51-70', '70+']
df['medal_grouping'] = pd.cut(df['total'],
                              bins=[-1, 0, 5, 10, 15, 20, 30, 40, 50, 70, 140],
                              labels=bin_labels)
df['numerical_medal_grouping'] = pd.cut(df['total'],
                              bins=[-1, 0, 5, 10, 15, 20, 30, 40, 50, 70, 140],
                              labels=range(10))

In [44]:
df = df.set_index("country_name",drop=True)
new_df = df.rename(columns={"total": "count_of_medals"})
new_df.head()

,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,count_of_medals,medal_grouping,numerical_medal_grouping
country_name,,,,,,,,
Afghanistan,33370794,613.856689,0.465,0.676,12,1,1-5,1
Albania,2889104,4578.631994,0.733,0.267,33,0,0,0
United Arab Emirates,9214175,43751.838890,0.835,0.247,70,0,0,0
Argentina,42669500,12334.798250,0.836,0.364,34,4,1-5,1
Armenia,2912403,3986.231624,0.733,0.299,37,2,1-5,1


### Split our preprocessed data into our features and target arrays

In [46]:
# Create our target
y = new_df["numerical_medal_grouping"].values
y[:20]

[1, 0, 0, 1, 1, ..., 1, 0, 4, 0, 4]
Length: 20
Categories (10, int64): [0 < 1 < 2 < 3 ... 6 < 7 < 8 < 9]

In [47]:
# Optimizing and transforming features
# Transforming CPI to values between 0-1
new_df["corruption_perceptions_index"] = new_df["corruption_perceptions_index"] / 100
# Transforming GII to invert values. Higher values will now correspond to more gender equality.
new_df["gender_inequality_index"] = 1 - new_df["gender_inequality_index"]
# Replacing GDP per capita with total GDP
new_df["gdp_total"] = new_df["population"] * new_df["gdp_per_capita"]
# X = new_df.drop(["count_of_medals","population","gdp_per_capita", "human_development_index", "gender_inequality_index", "corruption_perceptions_index"], axis=1)
new_df.head()

,population,gdp_per_capita,human_development_index,gender_inequality_index,corruption_perceptions_index,count_of_medals,medal_grouping,numerical_medal_grouping,gdp_total
country_name,,,,,,,,,
Afghanistan,33370794,613.856689,0.465,0.324,0.12,1,1-5,1,2.048489e+10
Albania,2889104,4578.631994,0.733,0.733,0.33,0,0,0,1.322814e+10
United Arab Emirates,9214175,43751.838890,0.835,0.753,0.70,0,0,0,4.031371e+11
Argentina,42669500,12334.798250,0.836,0.636,0.34,4,1-5,1,5.263197e+11
Armenia,2912403,3986.231624,0.733,0.701,0.37,2,1-5,1,1.160951e+10


In [48]:
# Create our features
X = new_df.drop(["count_of_medals", "medal_grouping", "numerical_medal_grouping","gdp_per_capita"], axis=1).values
X

array([[3.33707940e+07, 4.65000000e-01, 3.24000000e-01, 1.20000000e-01,
        2.04848851e+10],
       [2.88910400e+06, 7.33000000e-01, 7.33000000e-01, 3.30000000e-01,
        1.32281440e+10],
       [9.21417500e+06, 8.35000000e-01, 7.53000000e-01, 7.00000000e-01,
        4.03137100e+11],
       [4.26695000e+07, 8.36000000e-01, 6.36000000e-01, 3.40000000e-01,
        5.26319674e+11],
       [2.91240300e+06, 7.33000000e-01, 7.01000000e-01, 3.70000000e-01,
        1.16095129e+10],
       [2.34756860e+07, 9.35000000e-01, 8.77000000e-01, 8.00000000e-01,
        1.46748371e+12],
       [8.54635600e+06, 8.85000000e-01, 9.17000000e-01, 7.20000000e-01,
        4.41996132e+11],
       [9.53507900e+06, 7.51000000e-01, 6.70000000e-01, 2.90000000e-01,
        7.52442943e+10],
       [9.84429700e+06, 4.00000000e-01, 5.17000000e-01, 2.00000000e-01,
        2.70578327e+09],
       [1.12090570e+07, 8.90000000e-01, 9.24000000e-01, 7.60000000e-01,
        5.34678076e+11],
       [1.02868420e+07, 4.8000

In [49]:
# Splitting data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=15)

In [50]:
#Create a StandardScaler instances
scaler = MinMaxScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [51]:
# We need to convert our target labels (expected values) to categorical data
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
# Original label of `5` is one-hot encoded as `0000010000`
y_train[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 2 * number_input_features
hidden_nodes_layer2 = 2 * number_input_features

nn = tf.keras.models.Sequential()
number_input_features 

1

In [22]:
# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# # Second hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))
# After bucketing the medal count, you'd have to change this to have mulitiple outputs. 
# In other words, you'd need one node for each bucket. And then you'd change the activation to "softmax."
# You'd also have to one-hot encode your y values. 

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 4         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 4         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 3         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [24]:
# Compile the model
nn.compile(loss="mean_squared_error", optimizer="adam")

In [25]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

In [26]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=25,callbacks=[cp_callback])

Train on 119 samples
Epoch 1/25
119/119 [==============================] - 2s 14ms/sample - loss: 419.7206
Epoch 2/25
119/119 [==============================] - 0s 334us/sample - loss: 419.6590
Epoch 3/25
119/119 [==============================] - 0s 256us/sample - loss: 419.6039
Epoch 4/25
119/119 [==============================] - 0s 566us/sample - loss: 419.5429
Epoch 5/25
119/119 [==============================] - 0s 288us/sample - loss: 419.4847
Epoch 6/25
119/119 [==============================] - 0s 431us/sample - loss: 419.4285
Epoch 7/25
119/119 [==============================] - 0s 411us/sample - loss: 419.3600
Epoch 8/25
119/119 [==============================] - 0s 238us/sample - loss: 419.3132
Epoch 9/25
 32/119 [=======>......................] - ETA: 0s - loss: 74.6454
Epoch 00009: saving model to checkpoints/weights.09.hdf5
119/119 [==============================] - 0s 675us/sample - loss: 419.2428
Epoch 10/25
119/119 [==============================] - 0s 247us/sample - 

In [27]:
# # Evaluate the model using the test data
# model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [28]:
y_train_pred = nn.predict(X_train)
y_test_pred = nn.predict(X_test)

In [29]:
from sklearn.metrics import r2_score
r2_score(y_train, y_train_pred)

-0.17097429222045113

In [30]:
r2_score(y_test, y_test_pred)

-0.18587038462969518

In [31]:
y_test

array([ 0, 13,  1,  0,  1,  0,  0,  4,  0, 15,  3, 12,  0,  0,  0,  5,  1,
        0,  6, 17,  6,  2,  0,  0,  0,  0, 18,  0,  0, 12,  0,  0,  0,  7,
        0,  0, 50, 38,  0, 97])

In [33]:
y_test_pred

array([[0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552],
       [0.09259552]], dtype=float32)

### Saving the model

In [ ]:
# Export our model to HDF5 file
nn.save("DL_trained_model.h5")